In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import contractions
import demoji
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv('Training Data (TaskB).csv')

for i in range(len(train_df.axes[0])):
    words = train_df.iat[i,0][1:-1].split(',')
    
    text = []
    for word in words:
        word = word.strip()[1:-1]
        text.append(word)
        
    train_df.iat[i,0] = text
        
train_df

,text,label_category
0,"[lefti, sjw, simp, women, eat, basic, major, t...",4. prejudiced discussions
1,"[thank, lord, favor, whoso, findeth, wife, fin...",none
2,"[anoth, democrat, signal, eyeglass, way, quali...",2. derogation
3,"[met, girl, tinder, lost, felt, much, better, ...",none
4,"[tri, wit, group, teenag, girl, none, could, p...",none
...,...,...
11195,"[fuck, would, know, toxic, noth, pictur, first...",none
11196,"[illeg, immigr, stronger, togeth, deport, mom,...",none
11197,"[first, reaction, bulli, zoey, tuhr, shake, tr...",3. animosity
11198,"[end, time, birth, pain, get, closer, togeth, ...",none


In [3]:
test_df = pd.read_csv('Testing Data (TaskB).csv')

for i in range(len(test_df.axes[0])):
    words = test_df.iat[i,0][1:-1].split(',')

    text = []
    for word in words:
        word = word.strip()[1:-1]
        text.append(word)

    test_df.iat[i,0] = text
        
test_df

,text,label_category
0,"[damn, write, pretti, chaotic]",none
1,"[fuck, woman, still, mp]",none
2,"[also, enough, evid, like, goldberg, anti, gg,...",none
3,"[fuck, nine, mm, get, arab, fuck, monkey]","1. threats, plans to harm and incitement"
4,"[lefti, dad, offer, daughter, knock, maga, hat...",none
...,...,...
2795,"[would, american, pleas, stop, upset, presid, ...",none
2796,"[mind, stay, home, mom, never, lift, finger]",none
2797,"[know, come, revolut, much, guy, finish, child...",none
2798,"[black, girl, pure, breed]",none


In [4]:
df = pd.read_csv('Original Data.csv')
df = df.drop(['rewire_id'], axis=1)
df = df.drop(['label_sexist'], axis=1)
df

,text,label_category,label_vector
0,"Damn, this writing was pretty chaotic",none,none
1,"Yeah, and apparently a bunch of misogynistic v...",none,none
2,How the FUCK is this woman still an MP!!!???,none,none
3,Understand. Know you're right. At same time I ...,none,none
4,Surprized they didn't stop and rape some women,none,none
...,...,...,...
13995,complexes like the 'nice chicks' that go after...,2. derogation,2.1 descriptive attacks
13996,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",none,none
13997,Lol I imagine there would be simps that are li...,none,none
13998,"It's not, the girls I go on dates with don't k...",none,none


In [5]:
df['text'] = df['text'].str.replace('`', "'") #consistent quotes
df['text'] = df['text'].str.replace('“', '"') #consistent quotes
df['text'] = df['text'].str.replace('”', '"') #consistent quotes

for i in range(len(df.axes[0])):
    df.iat[i,0] = contractions.fix(df.iat[i,0]) #remove contractions

In [6]:
df['text'] = df['text'].str.replace('[0-9]*', "", regex=True) #remove digits
df['text'] = df['text'].str.replace('\\[USER\\]|\\[URL\\]', "", regex=True) #remove "URL" and "USER"
df['text'] = df['text'].str.lower() #convert to lowercase

In [7]:
df['text'] = df['text'].str.replace(r'\u200d', "", regex=True)
df['text'] = df['text'].str.replace(r'\u200f', "", regex=True)
df['text'] = df['text'].str.replace(r'\u200b', "", regex=True)

In [8]:
punctuation = string.punctuation + "—"
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i in range(len(df.axes[0])):
    text = df.iat[i,0]
    text = text.translate(str.maketrans(punctuation, ' '*len(punctuation), '')) #remove punctuations
    text = demoji.replace(text, "")
    text = word_tokenize(text) #tokenize
    text = [word for word in text if word not in stop_words] #remove stopwords
    text = [porter.stem(word) for word in text] #stemming
    df.iat[i,0] = text
    
df

,text,label_category,label_vector
0,"[damn, write, pretti, chaotic]",none,none
1,"[yeah, appar, bunch, misogynist, virgin, one, ...",none,none
2,"[fuck, woman, still, mp]",none,none
3,"[understand, know, right, time, know, enough, ...",none,none
4,"[surpriz, stop, rape, women]",none,none
...,...,...,...
13995,"[complex, like, nice, chick, go, bad, boy, nah...",2. derogation,2.1 descriptive attacks
13996,"[graphic, germani, muslim, refuge, stab, young...",none,none
13997,"[lol, imagin, would, simp, like, deserv, ugli,...",none,none
13998,"[girl, go, date, kiss, first, date, text, back...",none,none


In [9]:
def wordFrequencies(docs):
    all_words = []

    for doc in docs:
        for word in doc:
            all_words.append(word)

    all_words = nltk.FreqDist(all_words)
    return all_words

In [10]:
def kMostFrequentWords(k, all_words):
    words = list(all_words.keys())[:k]
    vocab = set()
    for word in words:
        vocab.add(word)

    vocab = {word:idx for idx, word in enumerate(vocab)}
    return vocab

In [11]:
def prepareTrueFalseFeatureSet(df, feature_words, size):
    feature_vector = []

    for i in range(len(df.axes[0])):
        features = {}
        text = df.iat[i,0]

        for feature_word in feature_words:
            features[feature_word] = feature_word in text

        feature_vector.append(features)
    
    return [(feature_vector[i], df.iat[i,1]) for i in range(len(feature_vector))]

In [12]:
def prepareCountFeatureSet(df, feature_words, size):
    feature_vector = []

    for i in range(len(df.axes[0])):
        features = {}
        text = df.iat[i,0]

        for feature_word in feature_words:
            features[feature_word] = 0
            for word in text:
                if word == feature_word:
                    features[word] += 1

        feature_vector.append(features)
    
    return [(feature_vector[i], df.iat[i,1]) for i in range(len(feature_vector))]

In [13]:
def sklearnKNNClassifier(num_neighbours, train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    knn = KNeighborsClassifier(n_neighbors=num_neighbours)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100, "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [14]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnKNNClassifier(5, prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))

#### scikit-learn k-NN Classifier (True/False Features Set)

Number of neighbours = 3, Size of Feature Vector = 500, Accuracy = 71.86, F1 (Macro): 24.51<br>
Number of neighbours = 3, Size of Feature Vector = 1000, Accuracy = 73.86, F1 (Macro): 27.98<br>
Number of neighbours = 3, Size of Feature Vector = 1500, Accuracy = 73.43, F1 (Macro): 25.77<br>
Number of neighbours = 3, Size of Feature Vector = 2000, Accuracy = 74.57, F1 (Macro): 25.67<br>
Number of neighbours = 3, Size of Feature Vector = 3000, Accuracy = 76.18, F1 (Macro): 24.59<br><br>

Number of neighbours = 7, Size of Feature Vector = 500, Accuracy = 75.61, F1 (Macro): 24.29<br>
Number of neighbours = 7, Size of Feature Vector = 1000, Accuracy = 75.93, F1 (Macro): 24.41<br>
Number of neighbours = 7, Size of Feature Vector = 1500, Accuracy = 75.68, F1 (Macro): 21.99<br>
Number of neighbours = 7, Size of Feature Vector = 2000, Accuracy = 75.36, F1 (Macro): 22.53<br>
Number of neighbours = 7, Size of Feature Vector = 3000, Accuracy = 76.07, F1 (Macro): 20.09<br><br>

Number of neighbours = 15, Size of Feature Vector = 500, Accuracy = 75.71, F1 (Macro): 21.10<br>
Number of neighbours = 15, Size of Feature Vector = 1000, Accuracy = 76.29, F1 (Macro): 19.93<br>
Number of neighbours = 15, Size of Feature Vector = 1500, Accuracy = 76.29, F1 (Macro): 18.02<br>
Number of neighbours = 15, Size of Feature Vector = 2000, Accuracy = 76.18, F1 (Macro): 18.09<br>
Number of neighbours = 15, Size of Feature Vector = 3000, Accuracy = 76.21, F1 (Macro): 17.72<br><br>


In [15]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnKNNClassifier(5, prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))

#### scikit-learn k-NN Classifier (Count Features Set)

Number of neighbours = 3, Size of Feature Vector = 500, Accuracy = 72.00, F1 (Macro): 24.21<br>
Number of neighbours = 3, Size of Feature Vector = 1000, Accuracy = 73.11, F1 (Macro): 26.11<br>
Number of neighbours = 3, Size of Feature Vector = 1500, Accuracy = 73.43, F1 (Macro): 25.67<br>
Number of neighbours = 3, Size of Feature Vector = 2000, Accuracy = 74.07, F1 (Macro): 24.25<br>
Number of neighbours = 3, Size of Feature Vector = 3000, Accuracy = 75.43, F1 (Macro): 23.91<br><br>

Number of neighbours = 7, Size of Feature Vector = 500, Accuracy = 75.07, F1 (Macro): 24.15<br>
Number of neighbours = 7, Size of Feature Vector = 1000, Accuracy = 75.89, F1 (Macro): 23.47<br>
Number of neighbours = 7, Size of Feature Vector = 1500, Accuracy = 76.11, F1 (Macro): 22.11<br>
Number of neighbours = 7, Size of Feature Vector = 2000, Accuracy = 76.18, F1 (Macro): 23.23<br>
Number of neighbours = 7, Size of Feature Vector = 3000, Accuracy = 76.43, F1 (Macro): 21.31<br><br>

Number of neighbours = 15, Size of Feature Vector = 500, Accuracy = 77.11, F1 (Macro): 55.59<br>
Number of neighbours = 15, Size of Feature Vector = 1000, Accuracy = 78.75, F1 (Macro): 55.54<br>
Number of neighbours = 15, Size of Feature Vector = 1500, Accuracy = 76.75, F1 (Macro): 48.43<br>
Number of neighbours = 15, Size of Feature Vector = 2000, Accuracy = 77.18, F1 (Macro): 49.02<br>
Number of neighbours = 15, Size of Feature Vector = 3000, Accuracy = 76.50, F1 (Macro): 46.29<br><br>

In [16]:
def sklearnLinearSVC(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = LinearSVC(fit_intercept=False, loss='hinge', max_iter=5000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [17]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnLinearSVC(prepareTrueFalseFeatureSet(train_df, vocab, 1000), prepareTrueFalseFeatureSet(test_df, vocab, 1000))

#### scikit-learn Linear SVC (True/False Features Set)

With feature vector of size 500, Accuracy: 72.29, F1 (Macro): 25.52<br>
With feature vector of size 1000, Accuracy: 73.64, F1 (Macro): 28.90<br>
With feature vector of size 1500, Accuracy: 72.86, F1 (Macro): 30.18<br>
With feature vector of size 2000, Accuracy: 72.61, F1 (Macro): 30.95<br>
With feature vector of size 2500, Accuracy: 72.18, F1 (Macro): 31.95<br>
With feature vector of size 3000, Accuracy: 72.32, F1 (Macro): 32.22<br>

In [18]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnLinearSVC(prepareCountFeatureSet(train_df, vocab, 1000), prepareCountFeatureSet(test_df, vocab, 1000))

#### scikit-learn Linear SVC (Count Features Set)

With feature vector of size 500, Accuracy: 73.93, F1 (Macro): 25.69<br>
With feature vector of size 1000, Accuracy: 73.96, F1 (Macro): 28.86<br>
With feature vector of size 1500, Accuracy: 72.93, F1 (Macro): 29.47<br>
With feature vector of size 2000, Accuracy: 72.86, F1 (Macro): 30.13<br>
With feature vector of size 2500, Accuracy: 72.61, F1 (Macro): 31.48<br>
With feature vector of size 3000, Accuracy: 72.00, F1 (Macro): 31.90<br>

In [19]:
def sklearnDecisionTree(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = DecisionTreeClassifier(max_depth=7)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [20]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnDecisionTree(prepareTrueFalseFeatureSet(train_df, vocab, 1000), prepareTrueFalseFeatureSet(test_df, vocab, 1000))

#### scikit-learn Decision Tree Classifier (True/False Features Set)

##### max_depth = 7

With feature vector of size  500, Accuracy: 78.07, F1 (Macro): 25.59<br>
With feature vector of size 1000, Accuracy: 79.39, F1 (Macro): 29.17<br>
With feature vector of size 1500, Accuracy: 79.43, F1 (Macro): 29.24<br>
With feature vector of size 2000, Accuracy: 79.43, F1 (Macro): 29.26<br>
With feature vector of size 3000, Accuracy: 79.57, F1 (Macro): 29.60<br>

In [21]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnDecisionTree(prepareCountFeatureSet(train_df, vocab, 1000), prepareCountFeatureSet(test_df, vocab, 1000))

#### scikit-learn Decision Tree Classifier (Count Features Set)

##### max_depth = 7

With feature vector of size  500, Accuracy: 78.25, F1 (Macro): 26.52<br>
With feature vector of size 1000, Accuracy: 79.43, F1 (Macro): 29.29<br>
With feature vector of size 1500, Accuracy: 79.50, F1 (Macro): 29.43<br>
With feature vector of size 2000, Accuracy: 79.50, F1 (Macro): 29.43<br>
With feature vector of size 3000, Accuracy: 79.64, F1 (Macro): 29.77<br>

In [22]:
def sklearnRandomForest(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = RandomForestClassifier(n_estimators=100)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred), "F1 Macro:", metrics.f1_score(y_test, y_pred, average="macro"))

In [23]:
# vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
# sklearnRandomForest(prepareTrueFalseFeatureSet(train_df, vocab, 1000), prepareTrueFalseFeatureSet(test_df, vocab, 1000))

#### scikit-learn Random Forest Classifier (True/False Features Set)

##### num_estimators = 100

With feature vector of size 500, Accuracy: 76.32, F1 (Macro): 28.56<br>
With feature vector of size 1000, Accuracy: 78.29, F1 (Macro): 30.84<br>
With feature vector of size 1500, Accuracy: 78.64, F1 (Macro): 31.50<br>
With feature vector of size 2000, Accuracy: 78.57, F1 (Macro): 32.86<br>
With feature vector of size 3000, Accuracy: 79.11, F1 (Macro): 31.90<br>
With feature vector of size 4000, Accuracy: 79.46, F1 (Macro): 32.53<br>
With feature vector of size 5000, Accuracy: 79.61, F1 (Macro): 32.06<br>

In [24]:
# vocab = kMostFrequentWords(1000, wordFrequencies(train_df['text']))
# sklearnRandomForest(prepareCountFeatureSet(train_df, vocab, 1000), prepareCountFeatureSet(test_df, vocab, 1000))

#### scikit-learn Random Forest Classifier (Count Features Set)

##### num_estimators = 100

With feature vector of size 500, Accuracy: 76.46, F1 (Macro): 28.83<br>
With feature vector of size 1000, Accuracy: 78.61, F1 (Macro): 31.48<br>
With feature vector of size 1500, Accuracy: 78.68, F1 (Macro): 32.32<br>
With feature vector of size 2000, Accuracy: 79.07, F1 (Macro): 33.28<br>
With feature vector of size 3000, Accuracy: 79.29, F1 (Macro): 32.63<br>
With feature vector of size 4000, Accuracy: 79.64, F1 (Macro): 31.93<br>
With feature vector of size 5000, Accuracy: 79.61, F1 (Macro): 32.17<br>

In [25]:
def sklearnMultinomialNB(train_featuresets, test_featuresets):
    X_train = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in train_featuresets]
    y_train = [feature_tag_tuple[1] for feature_tag_tuple in train_featuresets]
    X_test = [list(feature_tag_tuple[0].values()) for feature_tag_tuple in test_featuresets]
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    clf = MultinomialNB()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    return metrics.accuracy_score(y_test, y_pred), metrics.f1_score(y_test, y_pred, average="macro")

In [26]:
# interval = int(len(df)*0.2)

# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     print("\nSize: ", size)
    
#     acc_sum = 0
#     f1_sum = 0
    
#     for i in range(5):
#         train_df = pd.concat([df[0:i*interval], df[(i+1)*interval:len(df)]], axis=0)
#         test_df = df[i*interval:(i+1)*interval]
    
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         acc, f1 = sklearnMultinomialNB(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))
#         print("Accuracy: ", acc, ", F1 Score: ", f1)
#         acc_sum += acc
#         f1_sum += f1
        
#     print("Average Accuracy: ", acc_sum/5, ", Average F1 Score: ", f1_sum/5)

# sklearnMultinomialNB(prepareTrueFalseFeatureSet(train_df, 1000), prepareTrueFalseFeatureSet(test_df, 1000))

#### scikit-learn Multinomial Naive Bayes (True/False Features Set)

With feature vector of size  500, Accuracy: 77.88, F1 (Macro): 29.62<br>
With feature vector of size 1000, Accuracy: 77.80, F1 (Macro): 31.35<br>
With feature vector of size 1500, Accuracy: 77.72, F1 (Macro): 31.98<br>
With feature vector of size 2000, Accuracy: 77.65, F1 (Macro): 32.54<br>
With feature vector of size 2500, Accuracy: 77.64, F1 (Macro): 31.67<br>
With feature vector of size 3000, Accuracy: 77.56, F1 (Macro): 31.31<br>

In [27]:
# interval = int(len(df)*0.2)

# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     print("\nSize: ", size)
    
#     acc_sum = 0
#     f1_sum = 0
    
#     for i in range(5):
#         train_df = pd.concat([df[0:i*interval], df[(i+1)*interval:len(df)]], axis=0)
#         test_df = df[i*interval:(i+1)*interval]
    
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         acc, f1 = sklearnMultinomialNB(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))
#         print("Accuracy: ", acc, ", F1 Score: ", f1)
#         acc_sum += acc
#         f1_sum += f1
        
#     print("Average Accuracy: ", acc_sum/5, ", Average F1 Score: ", f1_sum/5)
    
# sklearnMultinomialNB(prepareCountFeatureSet(train_df, 1000), prepareCountFeatureSet(test_df, 1000))

#### scikit-learn Multinomial Naive Bayes (Count Features Set)

With feature vector of size  500, Accuracy: 77.45, F1 (Macro): 30.16<br>
With feature vector of size 1000, Accuracy: 77.37, F1 (Macro): 32.45<br>
With feature vector of size 1500, Accuracy: 77.15, F1 (Macro): 33.01<br>
With feature vector of size 2000, Accuracy: 77.15, F1 (Macro): 32.96<br>
With feature vector of size 2500, Accuracy: 76.99, F1 (Macro): 32.54<br>
With feature vector of size 3000, Accuracy: 77.05, F1 (Macro): 31.89<br>

In [28]:
def nltkMultinomialNB(train_featuresets, test_featuresets):
    clf = nltk.NaiveBayesClassifier.train(train_featuresets)
    
    preds = []
    tests = [test_featuresets[i][0] for i in range(len(test_featuresets))]
    for test in tests:
        preds.append(clf.classify(test))
    
    y_test = [feature_tag_tuple[1] for feature_tag_tuple in test_featuresets]
    
    cnt = 0
    for i in range(len(preds)):
        if preds[i] == y_test[i]:
            cnt += 1
        
    return cnt/len(preds), metrics.f1_score(y_test, preds, average="macro")

In [29]:
# interval = int(len(df)*0.2)

# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     print("\nSize: ", size)
    
#     acc_sum = 0
#     f1_sum = 0
    
#     for i in range(5):
#         train_df = pd.concat([df[0:i*interval], df[(i+1)*interval:len(df)]], axis=0)
#         test_df = df[i*interval:(i+1)*interval]
    
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         acc, f1 = nltkMultinomialNB(prepareTrueFalseFeatureSet(train_df, vocab, size), prepareTrueFalseFeatureSet(test_df, vocab, size))
#         print("Accuracy: ", acc, ", F1 Score: ", f1)
#         acc_sum += acc
#         f1_sum += f1
        
#     print("Average Accuracy: ", acc_sum/5, ", Average F1 Score: ", f1_sum/5)
    
# nltkMultinomialNB(prepareTrueFalseFeatureSet(train_df, 1000), prepareTrueFalseFeatureSet(test_df, 1000))

#### NLTK Multinomial Naive Bayes (True/False Features Set)

With feature vector of size  500, Accuracy: 77.06, F1 (Macro): 31.19<br>
With feature vector of size 1000, Accuracy: 77.34, F1 (Macro): 34.74<br>
With feature vector of size 1500, Accuracy: 76.74, F1 (Macro): 34.82<br>
With feature vector of size 2000, Accuracy: 76.62, F1 (Macro): 34.30<br>
With feature vector of size 2500, Accuracy: 76.54, F1 (Macro): 33.76<br>
With feature vector of size 3000, Accuracy: 76.55, F1 (Macro): 33.50<br>

In [30]:
# interval = int(len(df)*0.2)

# for size in [500, 1000, 1500, 2000, 2500, 3000]:
#     print("\nSize: ", size)
    
#     acc_sum = 0
#     f1_sum = 0
    
#     for i in range(5):
#         train_df = pd.concat([df[0:i*interval], df[(i+1)*interval:len(df)]], axis=0)
#         test_df = df[i*interval:(i+1)*interval]
    
#         vocab = kMostFrequentWords(size, wordFrequencies(train_df['text']))
#         acc, f1 = nltkMultinomialNB(prepareCountFeatureSet(train_df, vocab, size), prepareCountFeatureSet(test_df, vocab, size))
#         print("Accuracy: ", acc, ", F1 Score: ", f1)
#         acc_sum += acc
#         f1_sum += f1
        
#     print("Average Accuracy: ", acc_sum/5, ", Average F1 Score: ", f1_sum/5)

# nltkMultinomialNB(prepareCountFeatureSet(train_df, 1000), prepareCountFeatureSet(test_df, 1000))

#### NLTK Multinomial Naive Bayes (Count Features Set)

With feature vector of size  500, Accuracy: 76.49, F1 (Macro): 30.42<br>
With feature vector of size 1000, Accuracy: 76.32, F1 (Macro): 32.00<br>
With feature vector of size 1500, Accuracy: 75.79, F1 (Macro): 30.88<br>
With feature vector of size 2000, Accuracy: 75.82, F1 (Macro): 31.00<br>
With feature vector of size 2500, Accuracy: 75.84, F1 (Macro): 30.35<br>
With feature vector of size 3000, Accuracy: 75.65, F1 (Macro): 29.48<br>